In [ ]:
from attack_correlation import *
from data_loader import data_load_into_graph as load
from utils import save_graph, load_graph

G, data = load()
# This is a bit pointless rn but works: save_graph(G, "cached_graphs/data_graph.pkl")
# G = load_graph("cached_graphs/data_graph.pkl")

In [ ]:
#H = attack_correlation(G, data)

#save_graph(H, "cached_graphs/attack_correlation_graph.pkl")
H = load_graph("cached_graphs/attack_correlation_graph.pkl")

In [ ]:
# Example of component (I broke this cell somehow and don't know how yet...)

import matplotlib.pyplot as plt

# Extract largest component
largest_cc = list(nx.connected_components(H.to_undirected()))[123]
H_sub = H.subgraph(largest_cc)

# Get node severities and scale them (e.g., severity 1–10 becomes size 300–1000)
node_severities = nx.get_node_attributes(H_sub, "severity")
min_sev, max_sev = min(node_severities.values()), max(node_severities.values())
node_sizes = [
  30 + 1000 * ((node_severities[n] - min_sev) / (max_sev - min_sev)) if max_sev > min_sev else 500
  for n in H_sub.nodes()
]

# Node labels: use event description
node_labels = {
    node: H_sub.nodes[node].get("description", "") 
    for node in H_sub.nodes
}

# Edge widths scaled from weights
raw_weights = [d.get("weight", 0) for _, _, d in H_sub.edges(data=True)]
min_w, max_w = min(raw_weights), max(raw_weights)
edge_weights = [
  2 + 5 * ((w - min_w) / (max_w - min_w)) if max_w > min_w else 5
  for w in raw_weights
]

plt.figure(figsize=(12, 8))
pos = nx.circular_layout(H_sub)

nx.draw(
    H_sub, pos,
    labels=node_labels,
    with_labels=True,
    node_size=node_sizes,
    node_color='lightblue',
    edge_color='gray',
    width=edge_weights
)

plt.show()

ValueError: min() arg is an empty sequence

In [8]:
from factor_graph import *

alerts = data['events'][:3]
fg = FactorGraph(alerts)

m = Messages()
# v = fg.variables["Collection"]
# print(m.marginal(v))
# v = fg.variables["Credential Access"]
# print(m.marginal(v))
print(m.marginals(fg))

{'Initial Access': array([0.06306674, 0.93693326]), 'Privilege Escalation': array([0.03369119, 0.96630881]), 'Collection': array([0.33029961, 0.66970039])}
